### Set up API key

In [13]:
# Import the Python SDK
import google.generativeai as genai # type: ignore
import json
import os

# Load the JSON file
config_file = "config.json"
if os.path.exists(config_file):
    with open(config_file, "r") as file:
        config = json.load(file)
        GEMINI_API_KEY = config.get("GEMINI_API_KEY")
else:
    raise FileNotFoundError(f"Configuration file '{config_file}' not found.")

genai.configure(api_key=GEMINI_API_KEY)

### Initialize the Generative Model

In [14]:
model = genai.GenerativeModel('gemini-pro')

### Generate text

In [15]:
example_prompt="Write a haiku about Generative AI and it's limitless potential."
print("Example Prompt:\n"+example_prompt,"\n")
response = model.generate_content(example_prompt)
print("Output:")
print(response.text)

Example Prompt:
Write a haiku about Generative AI and it's limitless potential. 

Output:
Infinite realms dance,
Generative AI's brush,
Potential unbound.


## Phishing Prevention

#### Generate Phishing Emails

In [16]:
name="John"
surname="Doe"
email="john.doe@example.com"
business_unit="Sales"
team_name="B2B Sales"

#Create an approximate combination of phishing parameters
phishing_examples = [
    {"Reason": "Account Suspicious Activity", "Fake Link": "https://example.com/secure-login", "Created By": "Sam Sussy"},
    {"Reason": "Password Expiry Notification", "Fake Link": "https://example.com/reset-password", "Created By": "Sally Sneaky"},
    {"Reason": "Exclusive Training Webinar", "Fake Link": "https://example.com/join-webinar", "Created By": "Richard Rascal"},
    {"Reason": "Email Storage Full", "Fake Link": "https://example.com/manage-storage", "Created By": "Bernard Bandit"}
]

In [17]:
#Take a random reason
import random
random_pick = random.choice(phishing_examples)

#Formulate a prompt
prompt=f'''Write an email from {random_pick["Created By"]} to {name} {surname} from the {team_name} team on the following theme: {random_pick["Reason"]}.
He will have to click on this link : {random_pick["Fake Link"]}. Only write the body of this email.'''

print("Prompt:",prompt)

Prompt: Write an email from Bernard Bandit to John Doe from the B2B Sales team on the following theme: Email Storage Full.
He will have to click on this link : https://example.com/manage-storage. Only write the body of this email.


In [18]:
body = model.generate_content(prompt)
print(body.text)

Hi John,

Your B2B Sales team email storage is currently full. Please click on the following link to manage your storage:

https://example.com/manage-storage

Thanks,
Bernard Bandit


In [19]:
subject = model.generate_content("Write me the subject of this email:\n"+body.text)
print(subject.text)

B2B Sales Team Email Storage Full


#### Sending an email using your gmail account

In [20]:
from google.auth.transport.requests import Request # type: ignore
from google.oauth2.credentials import Credentials # type: ignore
from google_auth_oauthlib.flow import InstalledAppFlow # type: ignore
from googleapiclient.discovery import build # type: ignore
from googleapiclient.errors import HttpError # type: ignore

# If modifying these scopes, delete the file token.json.
SCOPES = [
    "https://www.googleapis.com/auth/gmail.send",
]

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

In [21]:
import base64
import os.path
from email.mime.text import MIMEText

# This function has a weird return, ChatGPT made it that way, I didn't really look into it but here it is
# It just creates an email object
def create_message(sender, to, subject, message_text):
    """Create a message for an email."""
    message = MIMEText(message_text)
    message["to"] = to
    message["from"] = sender
    message["subject"] = subject
    return {"raw": base64.urlsafe_b64encode(message.as_bytes()).decode()}

def send_message(service, user_id, message):
    """Send an email message."""
    try:
        sent_message = (
            service.users()
            .messages()
            .send(userId=user_id, body=message)
            .execute()
        )
        print(f"Message sent! Message ID: {sent_message['id']}")
    except HttpError as error:
        print(f"An error occurred: {error}")

In [22]:
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)

    # Create and send a test email
    sender = "me"  # 'me' refers to the authenticated user
    recipient = "proximus.notifications@gmail.com" #put an email recipient, can be yourself but "me" doesn't work as argument

    message = create_message(sender, recipient, subject.text, body.text)
    send_message(service, "me", message)

except HttpError as error:
    print(f"An error occurred: {error}")

Message sent! Message ID: 1945b4b5fa3c1c88
